# Perfect indistinguishability

Caratterizzazione alternativa (ma equivalente) della perfect secrecy.


## Esperimento di indistinguishability

PrivK<sup>eav</sup><sub>M,π</sub>

PrivK $\rightarrow$ crittografia chiave privata \
π $\rightarrow$ nome del cifrario $\rightarrow$ π = (Gen, E, D) \
eav $\rightarrow$ eavesdropper $\rightarrow$ attaccante più debole (può soltanto guardare il canale pubblico)\
M $\rightarrow$ Mallory, attaccante

1. M $\rightarrow$ A : x<sub>0</sub>, x<sub>1</sub> &emsp; <i>attaccante M invia due plaintext ad A</i>
2. A : k $\leftarrow$ Gen() &emsp; <i>A genera una chiave con l'algoritmo Gen() $\in$ π</i> \
    &emsp; b $\leftarrow$ {0, 1} &emsp; <i>A genera un bit <b>b</b> uniformemente random</i>
3. A $\rightarrow$ M : y $\leftarrow$ E<sub>k</sub>(x<sub>b</sub>) &emsp; <i>A invia ad M un ciphertext ottenuto da uno dei 2 plaintext (scelto in base a <b>b</b>)</i>
4. M : b' $\leftarrow$ ... &emsp; <i>M vuole calcolare <b>b'</b> in modo che <b>b'=b</b> (vuole indovinare <b>b</b>)</i>

L'esperimento ha successo (PrivK<sup>eav</sup><sub>M,π</sub> = 1) se <b>b'=b</b>

### Cifrario perfectly indistinguishable

π è <b>perfectly indistuinguishable</b> (e anche perfetto) se:

$\forall$ M : Pr(PrivK<sup>eav</sup><sub>M,π</sub> = 1) = 1/2 &emsp; <i><b>per ogni avversario</b> M , <b>la probabilità che l'esperimento abbia successo è 1/2</b> (anche il migliore avversario è costretto a tirare a indovinare come sua migliore strategia)</i>

Qualunque M vuol dire anche l'avversario con HW illimitato e tempo illimitato (irrealistico)